In [17]:
genes_df = pd.read_csv("/Users/caicai/Downloads/Thesis_caihuixin/data/interim/variants.csv", dtype=str)
genes_df.head(5)


,Gene,Consequence,Transcript,Variation ID,vcf
0,DDX11L1,upstream_gene_variant,ENSG00000223972,rs62635278,chr1\t11154\t.\tA\tC\t72.03\t.\tAC=2;AF=0.500;...
1,DDX11L2,upstream_gene_variant,ENSG00000290825,rs62635278,chr1\t11154\t.\tA\tC\t72.03\t.\tAC=2;AF=0.500;...
2,WASH7P,downstream_gene_variant,ENSG00000227232,rs62635278,chr1\t11154\t.\tA\tC\t72.03\t.\tAC=2;AF=0.500;...
3,DDX11L1,upstream_gene_variant,ENSG00000223972,rs62635279,chr1\t11167\t.\tC\tA\t75.03\t.\tAC=2;AF=0.500;...
4,DDX11L2,upstream_gene_variant,ENSG00000290825,rs62635279,chr1\t11167\t.\tC\tA\t75.03\t.\tAC=2;AF=0.500;...


In [16]:
#!/usr/bin/env python3
import vcfpy
import pandas as pd
import re

annotated_vcf = "/Users/caicai/THESIS/annotated_everything_chr1.vcf"
# 1) 读取注释后VCF，保存非注释行（以便我们手动处理INFO字符串）
raw_lines = []
with open(annotated_vcf, 'r') as f:
    for line in f:
        line = line.rstrip('\n')
        if not line.startswith('#'):
            raw_lines.append(line)

# 2) 用 vcfpy 解析（从注释里提取CSQ进行后续表格信息处理）
vcf_reader = vcfpy.Reader.from_path(annotated_vcf)

variant_data = []
line_index = 0

for record in vcf_reader:
    original_line = raw_lines[line_index]
    line_index += 1

    columns = original_line.split('\t')
    if len(columns) >= 8:
        info = columns[7]  # 第8列是INFO

        # 找到 "CSQ=" 起始位置
        idx_csq = info.find("CSQ=")
        if idx_csq != -1:
            # 若前面有分号，则也一起去除
            start_cut = idx_csq
            if start_cut > 0 and info[start_cut - 1] == ';':
                start_cut -= 1
            info = info[:start_cut].rstrip(';')  # 保留到CSQ前，并清理末尾分号

        columns[7] = info
        cleaned_line = '\t'.join(columns[:8])
    else:
        # 如果列数异常，就原样保留
        cleaned_line = original_line

    csq_info = record.INFO.get("CSQ")
    if csq_info:
        for csq in csq_info:
            fields = csq.split('|')
            consequence = fields[1]
            gene = fields[3]
            transcript = fields[4]
            variID = fields[17]
            variant_data.append({
                "Gene": gene,
                "Consequence": consequence,
                "Transcript": transcript,
                "Variation ID": variID,
                # 这里写入“只保留到SOR为止”的行
                "vcf": cleaned_line
            })

# 去重 & 写CSV
df = pd.DataFrame(variant_data)
df_deduplicated = df.drop_duplicates()

output_csv = "/Users/caicai/Downloads/Thesis_caihuixin/data/interim/variants.csv"
df_deduplicated.to_csv(output_csv, index=False)


In [18]:
genes_df.shape

(720685, 5)

In [19]:

genes = set(genes_df["Gene"])
genes

{'RNU7-179P',
 'GNAT2',
 'WDR3',
 'ITGA10',
 'TMCO2',
 'RAP1GAP',
 'MAP1LC3C',
 'MAST2',
 'LINC01361',
 'FMO11P',
 'RO60',
 'DAP3P1',
 'SNORD21',
 'TEDDM1',
 'MFFP1',
 'QRSL1P1',
 'C1orf232',
 'DDR2',
 'SMYD3-AS1',
 'RPA2',
 'CAPZB',
 'LYPD8',
 'FMO7P',
 'ANKRD34A',
 'SNHG28',
 'LINC00869',
 'RNU6-1089P',
 'MAP10',
 'ZBTB40-IT1',
 'DESI2',
 'HIPK1',
 'ARHGEF10L',
 'RNA5SP80',
 'PRR13P1',
 'HMGN2P18',
 'OR2T34',
 'OR2M5',
 'PTPN2P1',
 'ZBTB41',
 'FAM91A3P',
 'SNRPEP10',
 'LINC02814',
 'RNA5SP162',
 'WBP1LP6',
 'RNU6-304P',
 'CICP26',
 'USH2A-AS2',
 'LPAR3',
 'H3P2',
 'HPCA',
 'TRNP1',
 'OVGP1',
 'LEMD1',
 'SMYD3-IT1',
 'MIR6726',
 'L1TD1',
 'RNVU1-15',
 'DEPDC1-AS1',
 'RNU6-877P',
 'IGFN1',
 'RRAGC-DT',
 'NXNP1',
 'XRCC6P3',
 'VAV3',
 'PRDX6',
 'RPL6P3',
 'NRAS',
 'NKAIN1P1',
 'LINC01364',
 'FCF1P7',
 'LINC02815',
 'C1orf127',
 'RPL31P12',
 'UBE2V2P3',
 'DDX59-AS1',
 'MCRIP2P2',
 'CTNNBIP1',
 'ERRFI1-DT',
 'RASAL2-AS1',
 'REN',
 'RNA5SP56',
 'RNU1-132P',
 'GALE',
 'MIR30E',
 'DDAH1',
 '

In [ ]:
pubmed = pd.read_csv("/Users/caicai/Downloads/Thesis_caihuixin/results/entities_extracted.csv", dtype=str)
pmids = list(set(pubmed["pmids"]))

In [23]:
len(pmids)

189